In [1]:
# Workflow Agents in Google ADK: Itinerary-Driven Packing List Example
# Welcome 👋🏻 In this notebook, you'll learn how to use Workflow Agents in Google's Agent Development Kit (ADK)
# to chain together two LLM agents: one that creates a travel itinerary, and another that generates a packing list
# based on that itinerary. This mirrors how real travelers plan: first decide what you'll do, then pack accordingly!

In [2]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.0.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /home/codespace/.python/current/lib/python3.12/site-packages
Requires: authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-secret-manager, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-trace, opentelemetry-sdk, pydantic, python-dotenv, PyYAML, sqlalchemy, tzlocal, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [5]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# SequentialAgent: A specialized agent that chains together multiple sub-agents in a predefined sequence.
from google.adk.agents import SequentialAgent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

#  InMemorySessionService: A simple, in-memory service for managing conversation sessions. 
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# google_search: An ADK tool that provides agents with the ability to perform web searches, enabling them to retrieve up-to-date information.
from google.adk.tools import google_search

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

# IPython.display: Used for rendering rich content like Markdown in Jupyter notebooks.
from IPython.display import Markdown, display

In [6]:
# Itinerary Agent: Definition
# This agent is designed to create a detailed travel itinerary for a given destination and duration.
# It leverages the `google_search` tool to find up-to-date recommendations for attractions, restaurants, and activities.
# The `output_key="itinerary"` specifies that this agent's primary output will be accessible under the key "itinerary",
# which is crucial for passing this output to subsequent agents in a workflow.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, always include it as a Markdown hyperlink like [Item Name](URL). try to add as many hyperlinks as possible!
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
    output_key="itinerary"  # This agent's output will be stored with the key "itinerary".
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

`itinerary_agent` successfully defined with `google_search` tool.


In [7]:
# Packing List Agent: Definition
# This agent's primary role is to generate a comprehensive packing list.
# Crucially, its instruction includes a placeholder `{itinerary}`, which tells the agent to
# expect and use the output from a previous step (the itinerary_agent in this workflow).
# The `output_key="packing_list"` ensures that its final output is stored under this key.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Generates a packing list based on itinerary, destination, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        The user will provide their destination, trip duration, and a detailed itinerary (including planned activities, locations, and types of events for each day).
        Your task is to generate a complete, practical packing list tailored to the specific activities and locations in the itinerary.
        
        Itinerary: {itinerary} 

        **Thoroughly analyze the provided Itinerary, considering the specific activities, locations, and time of day for each entry.** Determine necessary clothing, gear, and items based *directly* on these plans. For example, if the itinerary mentions hiking, suggest hiking boots. If it mentions a fancy dinner, suggest formal wear.

        Organize the packing list by category (such as Clothing, Toiletries, Electronics, Documents, and Other Essentials).
        If you notice a special activity or requirement in the itinerary, include specific items for it and justify their inclusion based on the activity.
        If any information is missing, make reasonable assumptions based on the itinerary and destination—do not ask the user for more details unless absolutely necessary.
        
        **After the main categorized list, provide a dedicated 'Outfit Suggestions for Key Activities' section.** For 2-3 prominent activities from the itinerary, suggest a complete outfit including clothing, shoes, and any relevant accessories, explaining *why* it's suitable for that specific event.

        Present the complete output in a clear, easy-to-read markdown format. For example:

        #### Clothing
        - Comfortable walking shoes (for city tours and museum visits)
        - Lightweight jacket (for potentially cooler evenings in [Destination])
        - Dress for dinner at a nice restaurant (if formal dining is in itinerary)
        - Swimwear (if beach or pool is in itinerary)
        - ...

        #### Toiletries
        - Toothbrush and toothpaste
        - Sunscreen (for outdoor activities)
        - ...

        #### Electronics
        - Phone and charger
        - Plug adapter (if needed for [Destination])
        - ...

        #### Documents
        - Passport
        - Travel insurance (important for any trip abroad)
        - ...

        #### Other Essentials
        - Daypack for city tours
        - Reusable water bottle

        #### Outfit Suggestions for Key Activities
        *   **Day 1 - Morning (Tokyo National Museum):** Comfortable smart casual. E.g., Dark jeans/trousers, a layered top (blouse or stylish t-shirt), and comfortable, stylish walking shoes. A light scarf or jacket for cooler museum interiors.
        *   **Day 2 - Evening (Ninja Restaurant):** Fun, relaxed evening wear. E.g., A casual dress or nice shirt with dark pants. Comfortable shoes for easy movement.
        *   **Day 3 - Afternoon (Sushi Making Class):** Practical and comfortable. E.g., T-shirt, comfortable pants that allow movement, and closed-toe shoes. Avoid overly loose sleeves for hygiene and safety.

        Only ask clarifying questions if the itinerary is extremely vague or unclear. Otherwise, use your best judgment and generate a full, actionable packing list.
    """,
    output_key="packing_list"  # This agent's output will be stored with the key "packing_list".
)

In [8]:
# Sequential Agent: Definition
# The SequentialAgent allows you to define a workflow by chaining multiple sub-agents together.
# In this setup, the output of the first sub-agent (itinerary_agent) is automatically
# passed as input to the instruction of the next sub-agent (packing_list_agent)
# through the `{itinerary}` placeholder defined in the packing_list_agent's instruction.
wanderwise_agent = SequentialAgent(
    name="wanderwise_agent",
    sub_agents=[itinerary_agent, packing_list_agent],  # Defines the order of execution.
    description="Plans a travel itinerary and prepares a packing list based on activities planned."
)

In [9]:
# Import necessary modules for type hinting
from typing import Dict

# Modular Function for Agent Interaction (Corrected and Updated for multiple outputs)
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> Dict[str, str]:
    """
    Runs an interaction with an ADK agent (can be a single Agent or SequentialAgent)
    and returns a dictionary of its final text responses.
    Each response is keyed by the agent's 'name' (event.agent_name),
    or a sequential 'response_N' if the agent_name is not available.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        A dictionary where keys are the `agent_name` of the agent that produced the final response.
        Values are the full text responses.
        Returns an empty dictionary if no final response events are found.
    """
    # Use the provided session service if available, otherwise create a new one.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Collect all final responses
    all_final_responses = []
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            full_text = ''.join(part.text for part in event.content.parts if part.text)
            all_final_responses.append(full_text)
    return all_final_responses

print("Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.")

Modular function `run_adk_agent_interaction` successfully updated to use `event.agent_name` for keys.


In [10]:
# Set up the session and user input
APP_NAME = "wanderwise_app"
USER_ID = "default_user"
SESSION_ID = f"{APP_NAME}_default_session"

#  Create a single instance of the session service.
# Passing a shared instance (`common_session_service`) to `run_adk_agent_interaction`
# is good practice, especially if you had a database-backed session service.
# For InMemorySessionService, it ensures all sessions are managed within the same memory scope.
common_session_service = InMemorySessionService()

# Define a list of diverse travel planning test cases for our workflow agent.
# Each dictionary contains a descriptive label, and the user's input.
test_cases = [
    {
        "label": "Adventure & Nature Trip (Banff)",
        "input_text": "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography.",
    },
    {
        "label": "Culture & History Trip (Rome)",
        "input_text": "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine.",
    },
    {
        "label": "Beach & Relaxation Trip (Maldives)",
        "input_text": "I'm visiting the Maldives for 4 days in January for relaxation and some water activities. 25F.",
    },
    {
        "label": "City Break & Shopping (Paris)",
        "input_text": "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!",
    }
]

print("------ [starting workflow agent interactions] ------\n")

# Iterate through each test case and run the travel planning workflow.
# The 'wanderwise_agent' is the SequentialAgent which orchestrates the 
# 'itinerary_agent' followed by the 'packing_list_agent'.
# The 'run_adk_agent_interaction' helper function handles session creation and
# collects all final responses (itinerary and packing list) into a list.
for i, case in enumerate(test_cases):
    print(f"\n--- Example {i+1}: {case['label']} ---")
    print(f"User Input: \"{case['input_text']}\"")

    # Invoke the workflow agent using the modular helper function.
    # We pass the shared 'common_session_service' instance.
    workflow_outputs_list = await run_adk_agent_interaction(
        agent=wanderwise_agent,
        user_id="user"+f"_{i+1}",
        session_id="session"+f"_{i+1}",  # Unique session ID for each test case
        input_text=case["input_text"],
        app_name=APP_NAME,
        session_service=common_session_service # Pass the shared service instance
    )

    # Parse and Display Outputs from the list.
    if workflow_outputs_list:
        for j, text_response in enumerate(workflow_outputs_list):
            display(Markdown(text_response))
    else:
        print(f"No outputs received for Example {i+1}.")
    
    print(f"--- Example {i+1}: Workflow Agent Interaction Complete ---")


print("\n------ [all workflow agent interactions complete] ------")


------ [starting workflow agent interactions] ------


--- Example 1: Adventure & Nature Trip (Banff) ---
User Input: "I'm planning a 7-day trip to Banff National Park in August. I want to do moderate hiking and wildlife photography."


Okay, I can help you plan a 7-day trip to Banff National Park in August focusing on moderate hiking and wildlife photography! Here is a possible itinerary.

Okay, here's a possible 7-day itinerary for Banff National Park in August, keeping in mind moderate hiking and wildlife photography:

### Day 1: Arrival in Banff & Vermilion Lakes

*   **Morning:** Arrive at Calgary International Airport (YYC), pick up your rental car, and drive to Banff (approx. 1.5-hour drive). Check into your accommodation.
*   **Afternoon:** Explore the **Vermilion Lakes** ([see here](https://www.banfflakelouise.com/vermilion-lakes)). This is a great spot for reflections of Mount Rundle and possible bird sightings. ([11, 17])
*   **Evening:** Dinner at **The Bison Restaurant & Terrace** ([Banff Hospitality Collective](https://www.banffhospitality.com/our-restaurants/the-bison/)) known for its regional cuisine and views. ([14])

### Day 2: Johnston Canyon & Bow Valley Parkway

*   **Morning:** Hike the **Johnston Canyon Trail** ([AllTrails](https://www.alltrails.com/trail/canada/alberta/johnston-canyon-to-upper-falls)) to the Upper and Lower Falls. This is a popular spot, so start early. ([7, 11, 19, 20])
*   **Lunch:** Grab a casual lunch at the **Johnston Canyon Cafe & Market.**
*   **Afternoon:** Drive along the **Bow Valley Parkway** ([Nature TTL](https://www.naturettl.com/banff-wildlife-10-places-to-see-wildlife-in-the-national-park/)), a scenic route known for wildlife viewing opportunities. Keep an eye out for elk, deer, and possibly bears. ([8, 17, 20])
*   **Evening:** Enjoy dinner at the **Banff Ave Brewing Co.** ([Banff Hospitality Collective](https://www.banffhospitality.com/our-restaurants/banff-ave-brewing-co/)), a local brewery with a lively atmosphere. ([10])

### Day 3: Lake Louise & Plain of Six Glaciers

*   **Morning:** Arrive early at **Lake Louise** to avoid crowds. Hike the **Plain of Six Glaciers Trail** ([AllTrails](https://www.alltrails.com/trail/canada/alberta/plain-of-six-glaciers-trail)) for stunning views of the lake and surrounding glaciers. ([7, 12, 19])
*   **Lunch:** Have lunch at the **Plain of Six Glaciers Tea House** ([Banff National Park](https://www.pc.gc.ca/en/pn-np/ab/banff)), a historic teahouse at the end of the trail. ([12, 19])
*   **Afternoon:** Explore the shores of Lake Louise and take photos of the iconic Fairmont Chateau Lake Louise. ([11])
*   **Evening:** Dinner at **Fairview Restaurant** at the Chateau Lake Louise for a fine dining experience.

### Day 4: Moraine Lake & Larch Valley (Seasonal)

*   **Morning:** Visit **Moraine Lake**. Access is restricted to shuttle, Roam Transit, or commercial tour. Hike to **Larch Valley & Sentinel Pass** ([Dreamy Adventurers](https://dreamyadventurers.com/banff-in-august/)), known for its stunning views (best in late August/September when larches turn gold). ([2, 6, 12]) Note: This hike may require a minimum group size at certain times.
*   **Lunch:** Pack a picnic lunch to enjoy by Moraine Lake.
*   **Afternoon:** Hike around Moraine Lake, taking in the views.
*   **Evening:** Enjoy dinner at **Block Kitchen and Bar** ([Banff Hospitality Collective](https://www.banffhospitality.com/our-restaurants/block-kitchen-bar/)) in Banff, known for its tapas-style dishes.

### Day 5: Wildlife Spotting & Photography along Icefields Parkway

*   **Morning:** Drive north on the **Icefields Parkway** ([Dreamy Adventurers](https://dreamyadventurers.com/banff-in-august/)), stopping at scenic viewpoints like **Bow Lake** and **Peyto Lake** ([6]). Keep an eye out for wildlife along the way (bighorn sheep are often seen near the Icefields). ([8])
*   **Lunch:** Have a picnic lunch at one of the many scenic pull-offs along the Icefields Parkway.
*   **Afternoon:** Visit the **Columbia Icefield** ([Pursuit Collection](https://www.pursuitcollection.com/columbia-icefield/)), take the Ice Explorer tour onto the Athabasca Glacier ([5]).
*   **Evening:** Return to Banff and have dinner at **The Grizzly House** ([Banff Grizzly House](https://www.grizzlyhouse.com/)), a fondue restaurant for a unique dining experience. ([15])

### Day 6: Sulphur Mountain & Banff Gondola

*   **Morning:** Hike up **Sulphur Mountain** ([AllTrails](https://www.alltrails.com/trail/canada/alberta/sulphur-mountain-trail)) or take the **Banff Gondola** ([Banff Gondola](https://www.banfflakelouise.com/banff-gondola)) to the summit for panoramic views. ([2, 3, 7, 17, 18])
*   **Lunch:** Enjoy lunch at the **Sky Bistro** at the top of Sulphur Mountain.
*   **Afternoon:** Explore the summit of Sulphur Mountain, visit the interpretive centre, and take photos of the surrounding mountain ranges.
*   **Evening:** Relax at the **Banff Upper Hot Springs** ([Banff Sunshine Village](https://www.sunshinevillage.com/things-to-do/banff-upper-hot-springs)) for a soak with a view. Enjoy dinner at **1888 Chop House** ([Fairmont Banff Springs](https://www.fairmont.com/banff-springs/dining/1888-chop-house/)) at the Fairmont Banff Springs. ([10])

### Day 7: Departure

*   **Morning:** Enjoy a final breakfast in Banff. Consider **Stock Food & Drink** ([Fairmont Banff Springs](https://www.fairmont.com/banff-springs/dining/stock-food-and-drink/)) for a quick bite. ([4])
*   **Afternoon:** Drive back to Calgary International Airport (YYC) for your departure.

**Important Notes for August:**

*   **Book accommodations and tours in advance:** August is peak season, so book well ahead. ([2, 6])
*   **Purchase a Parks Canada Pass:** Required for entry to Banff National Park. ([6])
*   **Bear Safety:** Carry bear spray and know how to use it. Be aware of your surroundings and make noise while hiking. ([6])
*   **Shuttle/Transportation:** Plan transportation to Moraine Lake, as personal vehicles are not allowed. ([2, 6])
*   **Weather:** Be prepared for all types of weather. August can have warm days and cool nights. Pack layers. ([2])
*   **Wildlife:** Dawn and dusk are best for wildlife viewing. Bow Valley Parkway and Icefields Parkway are good spots. ([8, 17])
*   **Hiking:** Check trail conditions before heading out.

Have a fantastic trip to Banff!


Okay, here's a packing list tailored to your 7-day trip to Banff National Park in August, focusing on moderate hiking and wildlife photography, based on the provided itinerary:

#### Clothing

*   **Moisture-wicking base layers:** (2-3) Essential for hiking to keep you dry and comfortable.
*   **Hiking pants:** (1-2 pairs) Durable and comfortable for hiking. Convertible pants are a good option.
*   **Hiking shorts:** (1 pair) For warmer days on the trail.
*   **Long-sleeved shirts:** (2-3) For layering and sun protection.
*   **T-shirts:** (3-4) Lightweight and breathable for everyday wear.
*   **Fleece jacket or warm sweater:** (1) For cool evenings and layering.
*   **Insulated jacket:** (1) Even in August, nights can be cold, especially at higher elevations.
*   **Waterproof and windproof jacket:** (1) Essential for unpredictable mountain weather.
*   **Hiking socks:** (5-7 pairs) Moisture-wicking and blister-resistant.
*   **Casual socks:** (2-3 pairs) For evenings and non-hiking days.
*   **Underwear:** (7 pairs)
*   **Sleepwear:** Comfortable pajamas.
*   **Comfortable walking shoes:** (1 pair) For exploring towns and less strenuous activities.
*   **Hiking boots:** (1 pair) Sturdy, waterproof hiking boots with good ankle support are crucial for the trails.
*   **Dress pants or skirt/dress:** (1) For dinners at nicer restaurants like Fairview Restaurant or 1888 Chop House.
*   **Nice shirts or blouses:** (2) For evenings out.
*   **Swimsuit:** (1) For Banff Upper Hot Springs.

#### Toiletries

*   **Toothbrush, toothpaste, floss**
*   **Shampoo, conditioner, body wash**
*   **Sunscreen:** High SPF, essential for protecting against strong mountain sun.
*   **Lip balm with SPF:** Protect your lips from sun and wind.
*   **Insect repellent:** Mosquitoes and other insects can be prevalent, especially near water.
*   **After-sun lotion:** To soothe skin after sun exposure.
*   **Hand sanitizer:**
*   **Personal medications:**
*   **Pain relievers:** (Ibuprofen or acetaminophen)
*   **Blister treatment:** (Moleskin or blister pads)
*   **Other personal toiletries**

#### Electronics

*   **Camera:** (Mirrorless or DSLR) For wildlife and landscape photography.
*   **Camera lenses:** (Wide-angle, telephoto) To capture different types of shots.
*   **Extra camera batteries:** Crucial, as cold weather can drain batteries quickly.
*   **Memory cards:** Bring several, as you'll likely take many photos.
*   **Phone and charger**
*   **Portable power bank:** To keep your devices charged on long days.
*   **Binoculars:** For wildlife viewing.

#### Documents

*   **Passport/Driver's License**
*   **Parks Canada Pass:** Purchase in advance online.
*   **Flight/Rental car confirmations**
*   **Accommodation confirmations**
*   **Travel insurance information**
*   **Emergency contact information**

#### Other Essentials

*   **Daypack:** (20-30L) To carry essentials on hikes.
*   **Reusable water bottle:** Stay hydrated on hikes and throughout the day.
*   **Headlamp or flashlight:** Essential for early morning/late evening hikes or in case of getting caught out after dark.
*   **Bear spray:** Essential for safety while hiking. Know how to use it.
*   **Bear bell:** To alert wildlife to your presence.
*   **First-aid kit:** Include essentials like bandages, antiseptic wipes, pain relievers, and blister treatment.
*   **Sunglasses:** Protect your eyes from the bright mountain sun.
*   **Hat:** For sun protection and warmth.
*   **Gloves or mittens:** Even in August, it can get cold, especially at higher elevations.
*   **Scarf or neck gaiter:** For added warmth and protection from the elements.
*   **Snacks:** (Energy bars, trail mix) For hikes.
*   **Cash/Credit cards**
*   **Dry bag or waterproof cover:** To protect electronics and valuables from rain.
*   **Duct tape:** For gear repairs.
*   **Small notebook and pen:** For taking notes or jotting down wildlife sightings.
*   **Whistle:** For attracting attention in case of emergency.
*   **Trekking poles:** (Optional) For added stability on hikes.

#### Outfit Suggestions for Key Activities

*   **Day 3 - Plain of Six Glaciers Hike:**
    *   **Top:** Moisture-wicking t-shirt, fleece jacket, and waterproof/windproof jacket.
    *   **Bottom:** Hiking pants or convertible hiking pants.
    *   **Shoes:** Sturdy hiking boots.
    *   **Accessories:** Hat, sunglasses, sunscreen, backpack with water, snacks, bear spray, and a first-aid kit.

*   **Day 5 - Icefields Parkway Wildlife Spotting:**
    *   **Top:** Layered clothing (t-shirt, long-sleeved shirt, fleece).
    *   **Bottom:** Comfortable pants.
    *   **Shoes:** Comfortable walking shoes or light hiking boots.
    *   **Accessories:** Binoculars, camera with telephoto lens, hat, sunscreen.

*   **Day 6 - Evening at Banff Upper Hot Springs & 1888 Chop House:**
    *   **Hot Springs:** Swimsuit, flip-flops or sandals.
    *   **Dinner:** Dress pants/skirt, nice shirt/blouse, dress shoes or boots, jacket or sweater.


--- Example 1: Workflow Agent Interaction Complete ---

--- Example 2: Culture & History Trip (Rome) ---
User Input: "I'm a 35 yo male, and I am going to Rome for 5 days in October. I'm very interested in ancient history, classical art, and local cuisine."


Okay, I can help you plan a 5-day trip to Rome focusing on ancient history, classical art, and local cuisine. Let's get started!

Ok, here is a possible 5-day itinerary for your trip to Rome in October, focusing on ancient history, classical art, and local cuisine. October in Rome offers pleasant weather, with average temperatures ranging from 15°C to 23°C (59°F to 73°F), but evenings can be cooler, so pack a light jacket. There's also a chance of rain.

### Day 1: Ancient Rome Immersion

*   **Morning:** Colosseum and Roman Forum. Explore the Colosseum, the iconic amphitheater of ancient Rome, and then wander through the ruins of the Roman Forum and Palatine Hill, the center of ancient Roman life. Book tickets in advance to avoid long lines. Consider a guided tour for in-depth historical and architectural insights.
    *   [Colosseum](https://www.getyourguide.com/rome-l33/colosseum-roman-forum-palatine-hill-priority-access-t19853/?cmp=ga&cq_src=google_ads&cq_cmp=17757441420&cq_con=140464775032&cq_term=colosseum%20tours&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=17757441420&adgroup_id=140464775032&target_id=kwd-298541472986&location_physical_id=9003730&match_type=b&ad_id=651109287339&msclkid=c8f9f83c79d11d8a1ef74112b283c822)
    *   [Roman Forum](https://www.getyourguide.com/rome-l33/rome-colosseum-ancient-rome-guided-walking-tour-t141870/?cmp=ga&cq_src=google_ads&cq_cmp=17757441420&cq_con=140464775032&cq_term=roman%20forum%20tours&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=17757441420&adgroup_id=140464775032&target_id=kwd-301871918389&location_physical_id=9003730&match_type=b&ad_id=651109287339&msclkid=e91b1d5479d11c6585ccf941d3506db5)
*   **Lunch:** A quick and tasty pizza al taglio (pizza by the slice) at Forno Campo de' Fiori [Forno Campo de' Fiori](https://www.facebook.com/Forno-Campo-de-Fiori-Roma-364707196999979/).
*   **Afternoon:** Pantheon and Piazza Navona. Visit the Pantheon, a marvel of Roman engineering, and then stroll through Piazza Navona, admiring the fountains and Baroque architecture.
    *   [Pantheon](https://www.getyourguide.com/rome-l33/rome-pantheon-priority-entrance-ticket-with-audio-guide-t456508/?cmp=ga&cq_src=google_ads&cq_cmp=19788349556&cq_con=146814412208&cq_term=pantheon%20rome&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=19788349556&adgroup_id=146814412208&target_id=kwd-1418946285&location_physical_id=9003730&match_type=e&ad_id=673962558280&msclkid=b5e2db6023201284f53c0945aa9a3081)
*   **Evening:** Dinner at Trattoria Perilli [Trattoria Perilli](https://www.trattoriaperilli.com/), known for classic Roman dishes like carbonara and amatriciana.

### Day 2: Vatican City and Classical Art

*   **Morning:** Vatican City: St. Peter's Basilica, Vatican Museums, and Sistine Chapel. Visit St. Peter's Basilica, a Renaissance masterpiece, and explore the Vatican Museums, including the Sistine Chapel with Michelangelo's frescoes. Book tickets online in advance and consider a guided tour to navigate the vast complex.
    *   [St. Peter's Basilica](https://www.getyourguide.com/vatican-city-l53/vatican-saint-peter-s-basilica-dome-climb-with-audio-guide-t460614/?cmp=ga&cq_src=google_ads&cq_cmp=19788349556&cq_con=146814412208&cq_term=st%20peter%27s%20basilica&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=19788349556&adgroup_id=146814412208&target_id=kwd-354985637&location_physical_id=9003730&match_type=e&ad_id=673962558280&msclkid=376972e22320119d372006457428a679)
    *   [Vatican Museums](https://www.getyourguide.com/vatican-city-l53/vatican-museums-sistine-chapel-st-peter-s-basilica-tour-t10349/?cmp=ga&cq_src=google_ads&cq_cmp=17757441420&cq_con=140464775032&cq_term=vatican%20museum%20tours&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=17757441420&adgroup_id=140464775032&target_id=kwd-300378335887&location_physical_id=9003730&match_type=b&ad_id=651109287339&msclkid=c115296a79d1123d015c5805f1a061ef)
*   **Lunch:** Pizza at L'Archetto [L'Archetto] - a great place to eat before heading to Vatican City
*   **Afternoon:** Capitoline Museums. Visit the Capitoline Museums, which house a world-renowned collection of ancient Roman art and artifacts, including sculptures like the Capitoline Wolf.
    *   [Capitoline Museums](http://www.museicapitolini.org/en/)
*   **Evening:** Dinner in Trastevere at Checco er Carettiere [Checco er Carettiere] - enjoy the authentic Roman cuisine.

### Day 3: Day Trip to Ostia Antica

*   **Morning:** Travel to Ostia Antica, the ancient port of Rome. Explore the well-preserved ruins of this ancient city, including houses, shops, and public buildings.
    *   [Ostia Antica](https://www.getyourguide.com/rome-l33/ostia-antica-guided-small-group-tour-t65731/?cmp=ga&cq_src=google_ads&cq_cmp=19788349556&cq_con=146814412208&cq_term=ostia%20antica%20tours&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=19788349556&adgroup_id=146814412208&target_id=kwd-306692521047&location_physical_id=9003730&match_type=e&ad_id=673962558280&msclkid=0a433c0623201412254c6615e6093b3c)
*   **Lunch:** Have lunch at a local restaurant in Ostia Antica.
*   **Afternoon:** Return to Rome.
*   **Evening:** Dinner at Armando al Pantheon [Armando al Pantheon] for traditional Roman cuisine near the Pantheon.

### Day 4: Art and Gardens

*   **Morning:** Borghese Gallery and Museum. Visit the Borghese Gallery, housed in the Villa Borghese, and admire masterpieces by Bernini and Caravaggio. Book tickets well in advance as entry is timed and limited. Afterwards, take a walk through the Villa Borghese gardens.
    *   [Borghese Gallery](https://www.getyourguide.com/rome-l33/borghese-gallery-and-gardens-skip-the-line-tour-t36669/?cmp=ga&cq_src=google_ads&cq_cmp=17757441420&cq_con=140464775032&cq_term=borghese%20gallery%20tours&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=17757441420&adgroup_id=140464775032&target_id=kwd-304012256791&location_physical_id=9003730&match_type=b&ad_id=651109287339&msclkid=b40b87d279d1147f035779b091065121)
*   **Lunch:** Enjoy lunch near the Borghese Gallery.
*   **Afternoon:** Explore the Baths of Diocletian, an excellent 2-3 hour visit before catching an outgoing train from Termini
*   **Evening:** Enjoy dinner in the Monti neighborhood.

### Day 5: Hidden Gems and Culinary Delights

*   **Morning:** Explore the Appian Way (Via Appia Antica), an ancient Roman road. You can walk or rent a bike to explore the catacombs and ancient ruins along the road.
    *   [Appian Way](https://www.getyourguide.com/rome-l33/from-rome-appian-way-6-hour-ebike-tour-t123944/?cmp=ga&cq_src=google_ads&cq_cmp=17757441420&cq_con=140464775032&cq_term=appian%20way%20tours&cq_med=&cq_plac=&cq_net=g&cq_pos=&cq_plt=gp&campaign_id=17757441420&adgroup_id=140464775032&target_id=kwd-300377352781&location_physical_id=9003730&match_type=b&ad_id=651109287339&msclkid=b4c7163679d1171b373431a692042a3b)
*   **Lunch:** Try a cooking class focused on Roman cuisine.
    *   [Cooking Class](https://www.vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH9coMCYx2mGqDowFn_l0uUB7zVHvsxSa-jYrIDECp59lPv4WyNYW5dCV9LmIpJA_D0y-sKh_j69QzJ46i47bRNDjbbTS4L3qC8tZoGaPAVeBM4lYMdmBuOytz4m29ojSyTBJqvWVesI9XPsfFQpJoJgqepzhLk7F-BoGNE-RGIZ88rAxfheUxUJ93hetVpBr8=)
*   **Afternoon:** Visit the Baths of Caracalla, impressive ruins of ancient Roman baths.
*   **Evening:** Enjoy a farewell dinner at a restaurant of your choice, perhaps trying a traditional Roman pasta dish like cacio e pepe.

**Restaurant Recommendations:**

*   **Da Enzo al 29 (Trastevere):** Known for its authentic Roman cuisine and cozy atmosphere.
*   **Roscioli Salumeria con Cucina (near Campo de' Fiori):** A popular spot for Roman cuisine.
*   **Armando al Pantheon (near the Pantheon):** A classic Roman restaurant with traditional dishes.

Have a fantastic trip to Rome!


...

Okay! Based on your 4-day Maldives itinerary focused on relaxation and water activities, here is a packing list to help you prepare:

#### Clothing

*   **Swimsuits:** 2-3 (essential for daily water activities and relaxation)
*   **Rashguard or Swim Shirt:** (for sun protection during snorkeling and water sports)
*   **Cover-up:** (sarong, light dress, or tunic to wear over your swimsuit)
*   **Shorts:** 2-3 pairs (casual wear for daytime activities)
*   **T-shirts/Tank tops:** 3-4 (lightweight and comfortable for warm weather)
*   **Sundresses/Casual Dresses:** 2-3 (for lunches, exploring, and evenings)
*   **Lightweight pants or skirt:** 1 (for visiting local islands and potentially more conservative areas)
*   **Evening outfit:** 1 dress or smart casual outfit (for the romantic beach dinner or nice restaurant in Malé)
*   **Underwear:** Enough for each day
*   **Socks:** A few pairs (mostly for travel days)
*   **Sleepwear**

#### Shoes

*   **Flip-flops or sandals:** (essential for the beach and casual wear)
*   **Water shoes:** (for snorkeling and water activities, especially around coral reefs)
*   **Walking shoes or comfortable sneakers:** (for exploring local islands and Malé)

#### Toiletries

*   **Sunscreen:** (high SPF, reef-safe is recommended to protect the coral reefs)
*   **After-sun lotion:** (to soothe skin after sun exposure)
*   **Insect repellent:** (especially important for evenings)
*   **Shampoo, conditioner, and body wash:** (travel-sized or purchase upon arrival)
*   **Toothbrush, toothpaste, and floss**
*   **Deodorant**
*   **Makeup:** (as desired, but keep it light)
*   **Hairbrush and hair ties**
*   **Hand sanitizer**
*   **Any personal medications**

#### Electronics

*   **Phone and charger**
*   **Camera and charger:** (consider a waterproof camera or phone case for water activities)
*   **Portable power bank:** (for charging devices on the go)
*   **E-reader or books:** (for relaxing on the beach)

#### Documents

*   **Passport**
*   **Visa:** (if required, check Maldives' visa requirements for your nationality)
*   **Flight/hotel confirmations**
*   **Travel insurance information**
*   **Copy of passport:** (keep separate from your actual passport)

#### Other Essentials

*   **Sunglasses**
*   **Hat or visor:** (for sun protection)
*   **Beach bag:** (for carrying essentials to the beach or pool)
*   **Snorkel gear:** (if you prefer your own, otherwise resorts often provide it)
*   **Dry bag:** (to protect electronics and valuables during water activities)
*   **Reusable water bottle:** (to stay hydrated and reduce plastic waste)
*   **Small backpack or daypack:** (for day trips)
*   **Cash:** (US dollars are widely accepted, but local currency (Maldivian Rufiyaa) might be useful on local islands)
*   **Ziploc bags:** (for wet clothing or small items)
*   **First-aid kit:** (band-aids, antiseptic wipes, pain relievers)

#### Outfit Suggestions for Key Activities

*   **Day 2 - Morning Snorkeling Trip:** Swimsuit, rashguard for sun protection, water shoes to protect your feet, and a waterproof camera to capture the underwater beauty.
*   **Day 3 - Exploring Malé:** Comfortable walking shoes, lightweight pants or a modest skirt (respectful for local culture), a breathable t-shirt, sunglasses, and a hat. A small backpack is useful for carrying water and souvenirs.
*   **Day 1 - Evening Romantic Beach Dinner:** Sundress or a nice blouse with linen pants/skirt, sandals, and light jewelry.

Have a fantastic trip to the Maldives!


--- Example 3: Workflow Agent Interaction Complete ---

--- Example 4: City Break & Shopping (Paris) ---
User Input: "I'm doing a 3-day city break in Paris in April. I want to visit famous landmarks, do some shopping, and enjoy cafes. Love Emily in Paris!"


Okay, here is a possible 3-day itinerary for your city break in Paris in April, keeping in mind your interest in famous landmarks, shopping, cafes, and the "Emily in Paris" vibe.

Here's your 3-day itinerary for Paris, designed to capture the essence of Parisian charm and "Emily in Paris" vibes:

### Day 1: Iconic Landmarks & Montmartre Magic

*   **Morning:** Start with the **Eiffel Tower** ([Eiffel Tower](https://www.toureiffel.paris/en)). Pre-book tickets to avoid long queues. Enjoy the panoramic views of Paris.
*   **Lunch:** Grab a casual lunch at a café near the Eiffel Tower. Consider **O Coffeeshop** for yummy treats like banana bread.
*   **Afternoon:** Head to **Montmartre**. Visit the **Sacré-Cœur Basilica** ([Sacre-Coeur Basilica](https://www.sacre-coeur-montmartre.com/en/)). Explore Place du Tertre, where artists create and display their work.
*   **Evening:** Dine at **La Maison Rose** ([La Maison Rose](https://www.lamaisonrosemontmartre.com/)), a charming and Instagrammable pink café in Montmartre, featured in "Emily in Paris".

### Day 2: Louvre, Gardens & Chic Shopping

*   **Morning:** Visit the **Louvre Museum** ([Louvre Museum](https://www.louvre.fr/en)). Focus on key highlights like the Mona Lisa and Venus de Milo. Book tickets in advance.
*   **Lunch:** Enjoy a coffee at **Café Kitsuné** near the **Palais Royal**.
*   **Afternoon:** Explore the **Jardin du Palais Royal** ([Jardin du Palais Royal](https://www.parismarais.com/en/info/palais-royal)). Then indulge in some shopping in the **Marais district**, known for its trendy fashion brands and concept stores.
*   **Evening:** Consider dinner at **Le Grand Véfour** ([Le Grand Véfour](https://www.legrandvefour.com/)), one of the oldest and most distinguished restaurants in Paris and an "Emily in Paris" filming location, if you're looking for a fancy experience.

### Day 3: Champs-Élysées, "Emily" Spots & Latin Quarter

*   **Morning:** Stroll along the **Champs-Élysées** ([Champs-Élysées](https://www.champselysees.fr/en/)). Visit the **Arc de Triomphe** ([Arc de Triomphe](http://www.paris-arc-de-triomphe.fr/)).
*   **Lunch:** Have lunch near **Place de l'Estrapade**, and visit **La Boulangerie Moderne** ([La Boulangerie Moderne](https://www.instagram.com/boulangeriemoderne/?hl=en)), for Emily's favorite pain au chocolat.
*   **Afternoon:** Explore the **Latin Quarter**. Visit the **Panthéon** ([Panthéon](http://www.paris-pantheon.fr/en/)).
*   **Evening:** Enjoy a final Parisian dinner at a traditional bistro in the Latin Quarter or consider a Seine River cruise for a different perspective of the city.

Enjoy your Parisian adventure!


Okay, here's a packing list tailored for your 3-day "Emily in Paris"-inspired city break in Paris in April:

#### Clothing
*   **Comfortable walking shoes:** Essential for all the walking and exploring. Consider stylish sneakers, loafers, or fashionable flats.
*   **Ankle Boots or Dressy Shoes:** For evenings and nicer dinners.
*   **Layers:** April in Paris can be unpredictable. Pack layers.
*   **Light jacket or trench coat:** A classic trench is perfect for Paris in April.
*   **Sweaters or cardigans:** For warmth and style.
*   **Tops:** A mix of stylish blouses, t-shirts, and a few dressier tops for evenings.
*   **Jeans or trousers:** Comfortable and versatile.
*   **Skirt or dress:** For a touch of Parisian chic.
*   **Socks:** Bring enough for each day.
*   **Underwear:** Pack enough for each day.
*   **Pajamas or sleepwear:** Comfortable for sleeping.
*   **Scarf:** A stylish accessory that can also provide warmth.

#### Toiletries
*   **Toothbrush, toothpaste, floss**
*   **Shampoo, conditioner, body wash**
*   **Makeup:** Whatever you usually use.
*   **Skincare:** Moisturizer, cleanser, etc.
*   **Sunscreen:** Even in April, protect your skin.
*   **Deodorant**
*   **Hairbrush/comb**
*   **Hair styling products**
*   **Perfume/Cologne:** Channel your inner Emily!
*   **Hand sanitizer**
*   **Makeup remover wipes**

#### Electronics
*   **Phone and charger**
*   **Portable charger:** To keep your phone charged while you're out exploring.
*   **Camera:** To capture all those Parisian moments (or use your phone).
*   **European plug adapter:** If needed for your devices.
*   **Headphones:** For listening to music or podcasts while traveling.

#### Documents
*   **Passport**
*   **Visa (if required)**
*   **Copies of passport and visa:** Keep separately from originals.
*   **Travel insurance details**
*   **Flight/train tickets**
*   **Hotel reservations**
*   **Tickets/reservations for attractions:** Eiffel Tower, Louvre, etc.
*   **Emergency contact information**

#### Other Essentials
*   **Daypack or crossbody bag:** For carrying essentials while exploring.
*   **Reusable water bottle:** Stay hydrated.
*   **Sunglasses**
*   **Small umbrella:** Paris weather can be unpredictable.
*   **Travel-sized tissues or wet wipes**
*   **Pain relievers (Ibuprofen, etc.)**
*   **Any necessary prescription medications**
*   **Lip balm**
*   **Snacks:** For when you need a quick energy boost.
*   **Phrasebook or translation app:** Useful for communicating.
*   **Small notebook and pen:** For jotting down notes or addresses.
*   **Credit cards and cash**

#### Outfit Suggestions for Key Activities
*   **Day 1 - Morning (Eiffel Tower):** Chic and comfortable. E.g., Jeans or stylish pants, a fitted sweater or blouse, a light jacket or trench coat, comfortable walking shoes or ankle boots, and a scarf. Accessorize with sunglasses and a stylish handbag.
*   **Day 2 - Evening (Le Grand Véfour - if chosen):** Dressy and elegant. E.g., A cocktail dress or a dressy skirt and top, heels or elegant flats, a clutch, and jewelry. A light wrap or jacket in case it's cool.
*   **Day 3 - Afternoon (Exploring Latin Quarter):** Casual and comfortable. E.g., Jeans or comfortable pants, a t-shirt or blouse, a cardigan or light jacket, comfortable walking shoes, and a crossbody bag.


--- Example 4: Workflow Agent Interaction Complete ---

------ [all workflow agent interactions complete] ------


In [11]:
# Congratulations 🎉 
# You’ve just built and run a SequentialAgent workflow in ADK that first creates an itinerary,
# then generates a packing list tailored to your actual plans.
# Try changing the user input or expanding the workflow with more steps!